In [62]:
def Hyp_param_list(Ind, Max):
    List = ['1', '4', '16', '32', '64']
    string = []
    Possible = List[min(4, Ind ) :min( 3 + Ind - int(Max/2), len(List)- 1)]
    if Ind == Max:
        return Possible
    else:
        Next = Hyp_param_list(Ind + 1, Max)
        return ['_'.join([j, i]) for i in Next for j in Possible]

In [67]:
li = Hyp_param_list(0, 0)

In [68]:
li

['1', '4', '16']

In [4]:
List = ['1', '2', '4', '8', '16', '32', '64']

In [5]:
List.extend(li)

In [6]:
List

['1', '2', '4', '8', '16', '32', '64', '64_64_64_64_64_64']

In [7]:
li.extend( Hyp_param_list(0, i+1 ) for i in range(3))

In [69]:
[li.extend(Hyp_param_list(0, i+1 )) for i in range(3)]

[None, None, None]

In [70]:
li

['1',
 '4',
 '16',
 '1_4',
 '4_4',
 '16_4',
 '1_16',
 '4_16',
 '16_16',
 '1_32',
 '4_32',
 '16_32',
 '1_4_16',
 '4_4_16',
 '1_16_16',
 '4_16_16',
 '1_4_32',
 '4_4_32',
 '1_16_32',
 '4_16_32',
 '1_4_16_32',
 '4_4_16_32',
 '1_16_16_32',
 '4_16_16_32',
 '1_4_32_32',
 '4_4_32_32',
 '1_16_32_32',
 '4_16_32_32']